# Stock Predict Model

## Import Modules

In [205]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# For Graph
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Yahoo API
import yfinance as yf

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Configurations

In [225]:
device = 'cpu' if torch.backends.mps.is_available() else 'cpu'
scaler = MinMaxScaler()
config = {
    'seed': 666999,
    'select_all': False,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 128,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'save_path': './models/stock.ckpt'
}

## Utility Functions

In [231]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            pred = scaler.inverse_transform(pred.detach().cpu())
            preds.append(pred)
    preds = torch.cat(preds, dim=0).numpy()
    return preds

## Dataset

In [207]:
class Stock_Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        self.y = y if y is None else torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Neural Network Model

In [208]:
class Stock_Model(nn.Module):
    def __init__(self, input_dim):
        super(Stock_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.Sigmoid(),
            nn.Linear(32, 16),
            nn.Sigmoid(),
            nn.Linear(16, 8),
            nn.Sigmoid(),
            nn.Linear(8, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

## Feature Selection

In [209]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

## Training Loop

In [211]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Dataloader

In [233]:
same_seed(config['seed'])
# re-index
index = ["Open", "High", "Low", "Close", "Volume", "Adj Close"]
# train_data size: 5255 x 7 (id + 6 features From 2000-01-01 to 2020-12-31)
# test_data size: 577 x 7 (id + 6 features From 2021-01-01 to 2023-12-31)
# train_data, test_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2020-12-31'))[index].values, \
#                         pd.DataFrame(yf.download('2330.TW', start='2021-01-01', end='2023-12-31'))[index[:-1]].values
raw_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2023-12-31'))
raw_data_scaled = scaler.fit_transform(raw_data[index].values)
train_data, test_data = train_test_split(
    raw_data_scaled,
    test_size=config['test_ratio'],
    random_state=config['seed']
)

train_data, valid_data = train_test_split(
    train_data,
    test_size=config['valid_ratio'],
    random_state=config['seed']
)

# train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])
# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = Stock_Dataset(x_train, y_train), \
                                             Stock_Dataset(x_valid, y_valid), \
                                             Stock_Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

[*********************100%***********************]  1 of 1 completed
train_data size: (3734, 6)
valid_data size: (934, 6)
test_data size: (1167, 6)
number of features: 3


## Training

In [213]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)

# with torch.profiler.profile(
#     activities=[torch.profiler.ProfilerActivity.CPU],
#     schedule = torch.profiler.schedule(
#         wait=0,
#         warmup=0,
#         active=1
#     ),
#     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
# ) as prof:
#     trainer(train_loader, valid_loader, model, config, device)
#     prof.step()

Epoch [1/3000]: 100%|██████████| 30/30 [00:00<00:00, 230.98it/s, loss=0.0458]

Epoch [1/3000]: Train loss: 0.0742, Valid loss: 0.0824
Saving model with loss 0.082...

Epoch [2/3000]: 100%|██████████| 30/30 [00:00<00:00, 229.55it/s, loss=0.0544]


Epoch [2/3000]: Train loss: 0.0739, Valid loss: 0.0812
Saving model with loss 0.081...


Epoch [3/3000]: 100%|██████████| 30/30 [00:00<00:00, 112.82it/s, loss=0.0833]


...

Epoch [536/3000]: Train loss: 0.0593, Valid loss: 0.0645


Epoch [537/3000]: 100%|██████████| 30/30 [00:00<00:00, 312.93it/s, loss=0.0525]


Epoch [537/3000]: Train loss: 0.0591, Valid loss: 0.0647


Epoch [538/3000]: 100%|██████████| 30/30 [00:00<00:00, 140.85it/s, loss=0.0437]


Epoch [538/3000]: Train loss: 0.0589, Valid loss: 0.0627

Model is not improving, so we halt the training session.


In [214]:
# print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))


In [215]:
%tensorboard --logdir=./log/

## Plot learning curves with tensorboard

In [216]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

## Testing

In [234]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: Expected 2D array, got 1D array instead:
array=[0.16865715 0.16859832 0.16859978 0.16760156 0.16840458 0.16752836
 0.16863844 0.16838759 0.16864917 0.16730243 0.16859135 0.16864535
 0.16856954 0.16839862 0.16860479 0.168161   0.1686559  0.16741762
 0.16848749 0.16745862 0.16866475 0.16863757 0.16841403 0.16777658
 0.16842631 0.1686419  0.1679565  0.16837063 0.16860235 0.16863903
 0.16840872 0.16860786 0.16754431 0.16847613 0.16810867 0.16841856
 0.16824779 0.16852632 0.16863394 0.16861638 0.16860822 0.16756156
 0.16863281 0.16860214 0.16860932 0.16851133 0.16849002 0.16864023
 0.16863951 0.16846114 0.16861477 0.16866425 0.16742352 0.16866359
 0.16821656 0.16847134 0.16865191 0.16841686 0.16844702 0.16819802
 0.16859904 0.16811442 0.16855034 0.16841918 0.16824594 0.16745409
 0.16867369 0.16805759 0.16857836 0.16854635 0.16840142 0.1686334
 0.16864181 0.1675176  0.16829607 0.16860521 0.16859967 0.16823819
 0.16867355 0.1686208  0.16805318 0.16857189 0.1685791  0.16779163
 0.1